Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 6.45 ms, sys: 348 µs, total: 6.8 ms
Wall time: 7.93 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

<Client: 'inproc://10.1.0.157/8139/1' processes=1 threads=6, memory=6.78 GiB>

1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 866 ms, sys: 126 ms, total: 992 ms
Wall time: 2.85 s


[1.7977999275253325,
 1.3397007813403028,
 1.808042155283469,
 1.5334102035572608,
 1.4603079442686657,
 1.2922083499070274,
 1.9798512682916405,
 1.5921352619890599,
 1.8128257004345176,
 1.5935193604646558,
 1.2013738902024562,
 1.5839986023334447,
 1.7067393298544258,
 1.6525515706808749,
 1.858924845042469,
 1.5506542452732646,
 1.2139504622964439,
 1.1231755251233961,
 1.1027667808749624,
 1.8125675611798133,
 1.475552683576491,
 1.0444124051940338,
 1.1753189287339847,
 1.2834155200535295,
 1.9504609697506528,
 1.5902036168830629,
 1.5282883957333728,
 1.9668254800706935,
 1.0067846549840873,
 1.1577123461115038,
 1.7740310137815685,
 1.2408877362118171,
 1.5409037631662592,
 1.6844022545018664,
 1.8855731080451585,
 1.3837231152243938,
 1.6993856074064544,
 1.1284873159372029,
 1.5760020263186953,
 1.7131428035013685,
 1.7724853420634696,
 1.2093194498350694,
 1.7889735086432994,
 1.5194280224467729,
 1.6158750124977168,
 1.0145704395926802,
 1.9259280547871958,
 1.1045526635006

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.8761367755097407,
 1.6156917220382632,
 1.855814789804529,
 1.5451775093259383,
 1.2903805897173315,
 1.0342363034032696,
 1.9727406648040495,
 1.2482909882483906,
 1.652325013420771,
 1.383785022053139,
 1.5433591603001167,
 1.8751425018376278,
 1.3006772713438268,
 1.6417390421101823,
 1.258638735821592,
 1.9604810169527198,
 1.7779224536436675,
 1.5175991023944093,
 1.9809731640276427,
 1.0257861192243443,
 1.134075847751093,
 1.727395676414353,
 1.4556194900054034,
 1.0550503560706095,
 1.131345367709352,
 1.2253588005367257,
 1.922221976104276,
 1.0821761917041832,
 1.048427729851797,
 1.7783211483485335,
 1.3662547751751777,
 1.2844046318234557,
 1.222658008598474,
 1.5744274727478311,
 1.2183268800949194,
 1.3701029862161147,
 1.3680075630877644,
 1.4345368722027183,
 1.8790755455324437,
 1.8711586853548245,
 1.8105243962095852,
 1.409495821510848,
 1.2981191710391116,
 1.0876341929032272,
 1.2993667700949985,
 1.065222296845762,
 1.001927402221397,
 1.3640687571312298,
 1.85

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 374 ms, sys: 15.4 ms, total: 389 ms
Wall time: 2.17 s
